In [ ]:
import seaborn as sns
sns.set()

In [ ]:
from static_grader import grader

# Image Classification with TensorFlow

Image classification is a common task for deep learning and neural networks.  The raw features coming in are the pixel values.  These are simple enough to deal with, but it is difficult to connect pixel values to determining whether an image is of a cat.  Older methods used a lot of clever filters, but the current best-of-breed algorithms simply throw a lot of linear algebra at the problem.

In this miniproject, you build a series of models to classify a series of images into one of ten classes. For expediency, these images are pretty small ($32\times32\times3$).  This can make classification a bit tricky&mdash;human performance is only about 94%.  Each of your models will be scored by comparing its accuracy to the accuracy of a reference model that we developed.  A score of 1 indicates that your model performs as well as the reference model; not that your accuracy is 100%!

You will be given both a training set and a validation set.  Ground truth values are provided for the training set.  You should train your models on this set, and then make predictions for each of the validation images.  These predictions will be submitted to the grader.

## A note on scoring

It **is** possible to score above 1 on these questions. This indicates that you've beaten our reference model&mdash;we compare our model's score on a test set to your score on a test set. See how high you can go!

## Downloading the data

We will be using the `CIFAR-10` data set.  It consists of 60,000 images, each $32\times32$ color pixels, each belonging to one of ten classes.  The following cell will download the data, in NumPy's `.npy` format.

In [ ]:
!aws s3 sync s3://dataincubator-course/cifar10/ .

We can load in the data like so:

In [ ]:
import numpy as np
import gzip

train_images = np.load(gzip.open('train_images.npy.gz', 'rb'))
train_labels = np.load(gzip.open('train_labels.npy.gz', 'rb'))
validation_images = np.load(gzip.open('validation_images.npy.gz', 'rb'))

The images are stored as four-dimensional arrays.  The first index indicates the image number, the second and third the $x$ and $y$ positions, and the fourth index the color channel.  Each pixel color is a floating point number between 0 and 1.  This convention allows us to view the images with matplotlib:

In [ ]:
import matplotlib.pyplot
matplotlib.pyplot.rcParams["axes.grid"] = False  #  Remove the grid lines from the image.
matplotlib.pyplot.imshow(train_images[0])

The classes have already been numbered 0-9 for us; those numbers are stored in the vector `train_labels`.

In [ ]:
train_labels[0]

The human-readable names associated with this classes are

In [ ]:
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
label_names[train_labels[0]]

So we can see that the image above is a frog.  (Now you see it!)

# Questions

## Perceptual Delta

Since we already have a number of labeled images, a simple approach would be to measure the difference between two images, and choose the label corresponding to nearby images.  To do this, we need to develop a metric to determine the distance between two images.  We'll make the simplifying (and completely wrong) assumption that this is just the average difference between the colors of the corresponding pixels in the two images.

While we could just take RMS difference of the red, green, and blue pixels, let's be slightly more sophisticated and look at human vision for a metric.  After all, we're pretty good at image classifications, so there might be some useful optimization here.

It turns out that modeling human perception is [extraordinarily complicated](https://en.wikipedia.org/wiki/Color_difference#CIEDE2000).  We're going to use a [simplified model](https://www.compuphase.com/cmetric.htm):

$$\Delta C \equiv \sqrt{2 \Delta R^2 + 4 \Delta G^2 + 3 \Delta B^2 + \bar R\left(\Delta R^2 - \Delta B^2 \right)} $$
where $(R_1, G_1, B_1)$ and $(R_2, G_2, B_2)$ are the RGB components of the two colors and
$$\begin{align}
\Delta R &= R_1 - R_2 \\
\Delta G &= G_1 - G_2 \\
\Delta B &= B_1 - B_2 \\
\bar R &= \textstyle\frac{1}{2}\left(R_1 + R_2\right)
\end{align}$$

This accounts for the fact that our eyes are most sensitive to green and least sensitive to red, and that perception is not constant with hue.

Build a graph that takes in a series of images, as well as a base image, and returns a result containing the $\Delta C$ value (for each pixel) between the base image and each image in the series.  

(Note that the intention here in our solution is that `images` will be a stack of images, while `base` is a single image.  Since the function is decorated with `tf.function` the computations within the function will be recorded in a computation graph and the output from this function will be a tensor.)

In [ ]:
def delta_func(images, base):
    dr = images[:, :, :, 0] - base[:, :, 0]
    dg = images[:, :, :, 1] - base[:, :, 1]
    db = images[:, :, :, 2] - base[:, :, 2]
    rmean = (images[:, :, :, 0] + base[:, :, 0])/2

    DeltaC = (2*(dr**2) + 4*(dg**2) + 3*(db**2) + rmean*((dr**2) - (db**2)))**.5
    return DeltaC

In [ ]:
deltas = delta_func(train_images, validation_images[0])

In [ ]:
distances = []
for i in range(50000):
    distances.append(np.mean(deltas[i]))

In [ ]:
indices = np.argsort(np.array(distances))[:100]

As stated, for two images, $I_1$ and $I_2$, we define the distance between $I_1$ and $I_2$ as the average $\Delta C$ value over the whole image, that is:

$$d(I_1, I_2) = \frac{1}{N}\sum_{p_j} \Delta C(p_j)$$
where the sum is over all pixels $p_j$, $\Delta C(p_2)$ is the $\Delta C$ value for the pixel $p_j$, and $N$ is the total number of pixels in each image ($N= 32\times 32$ in our case).

Using `delta_func` compute the distance between the first validation image and all of the training images.

**Checkpoint:** The mean value of the distances is 1.159, and the standard deviation of the distances is 0.182.

From these, find the 100 closest images from the training set to this image.  (Note that `numpy.argsort` might help here.)  Submit a list of the indices of these images to the grader.

In [ ]:
# indices = [43234]*100

grader.score('tf__perceptual_delta', indices)

**Extension:** What does this suggest about the proper class for this image?

> **Aside:** Essentially, we've started to implement a $k$-nearest neighbors algorithm, using this perceptual distance as our metric.  If we ran the difference between all of the validation images and each of the training images, we could make a prediction from the nearest images for each.  Give it a try, if you're interested, but this miniproject is going to go in another direction.

## Smallest delta model

Instead of comparing each validation image to each training image, let's generate a "typical" image for each class.  This typical image will be the one that minimizes the total (or average) perceptual delta from all training images in this class.  We could do that through some clever averaging, but let's treat it as a minimization problem so that we can do gradient descent.  

This problem will have two steps, the first step is to train a typical image for each class in the training set, the second will be to make predictions on the validation set.  These predictions will be based on the closest "typical" image trained in the first step.

Make the typical image a TensorFlow variable, starting with random pixel values.  It's this TensorFlow variable that we will train by performing gradient descent.

In [ ]:
import tensorflow as tf

In [ ]:
sample = tf.Variable(np.load(open('typical_6.npy','rb')))

Define an error function that measures the difference between that typical image and a set of images. You should be able to use the same `delta_func` you defined in the first problem.

In [ ]:
!aws s3 cp s3://dataincubator-course/miniprojects/tf/typical_6.npy .
matplotlib.pyplot.imshow(np.load(open('typical_6.npy','rb')))

In [ ]:
frogs = np.where(train_labels == 6)[0]   #  Get the indices for the frogs

fig, ax = matplotlib.pyplot.subplots(2, 2)
for i in range(4):
    np.ravel(ax)[i].imshow(train_images[ frogs[i] ])

matplotlib.pyplot.show()

Now, we will use gradient descent to find a typical image in order to minimize the error.  We want to do this for each of the ten classes of images, but let's first concentrate on how we do this for a single class (we can roll the code into a `for` loop later).  

First let's get the images that belong to a single class, say class 6.

In [ ]:
typical_ims = []
for i in range(10):
    img = tf.Variable(tf.random.uniform((32,32,3), dtype=tf.float64))
    label = i
    pics = np.where(train_labels == label)
    ims = train_images[pics]

    ##optimizer
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.07)
    with tf.GradientTape() as tape:
        loss = delta_func(ims, img)
    vars_l = [img]
    grads = tape.gradient(loss, vars_l)

    # Process the gradients, for example cap them, etc.
    # capped_grads = [MyCapper(g) for g in grads]
    #processed_grads = [process_gradient(g) for g in grads]

    # Ask the optimizer to apply the processed gradients.
    optimizer.apply_gradients(zip(grads, vars_l))
    typical_ims.append(img)

In [ ]:
typical_ims = [tf.cast(typical_ims[i], tf.float32) for i in range(10)]
typical_ims[1]

In [ ]:
typical_ims[1].numpy()

In [ ]:
diff = [c_delta(typical_ims[i],tf.cast(validation_images[0], tf.float32)).numpy() for i in range(10)]
np.argmin(np.asarray(diff))

In [ ]:
predicted_classes = []
for im in validation_images:
    im = tf.cast(im, tf.float32)
    diff = [c_delta(typical_ims[i],im).numpy() for i in range(10)]
    predicted_classes.append(np.argmin(np.asarray(diff)))

Now define the gradient descent optimizer, and a function that will calculate the gradient of the error with respect to your typical image, using the optimizer to apply the gradient to the typical image.  Remember that you need to specify a learning rate for the optimizer.  You might need to later experiment with the learning rate to find the right value so the training will converge to the minimum (and converge quickly).  (__Note:__  "Normal" learning rates are typically in the range 0.1 to 1, perhaps even smaller in some situations.  The learning rate we used in our solution was 10.)  

In [ ]:
len(predicted_classes)

Keeping in mind that our training set (for a single label) consists of 5,000 images, let's perform the training in batches.  To ensure good training, let's pick our batches randomly from the set of training images.  

Strictly speaking, we are performing a batched version of __stochastic gradient descent__.  

__Checkpoint:__  Here's the typical image we generated for label 6 when we did the training.  

You can compare this to the first four frog (class label 6) training images. A perfect fit, right?  

Having now done this training for one class, it should be easy to roll this code into a `for` loop to generate a typical image for each of the 10 classes.  Don't forget to save your typical image for each class, as they are needed for the prediction step.

**Hint:** We suggest storing your typical images in a file using the `numpy` "save" function.  That function allows you to save a `numpy` array to a binary file.  That way if you need to restart this notebook, you can reload your typical images you have already trained (not having to re-run the code above to create these typical images again).  (We used `numpy.load` near the start of this notebook to load the training and validation images that were stored in this binary format.)

Finally, we're ready to make a prediction for each validation image based on which typical image is nearest under our distance metric.  You need to generate a list of 10,000 integers, representing the predicted class of each validation image.   

__Hint:__ Use `delta_func` you defined from question 1 to find the distance between each validation image and a _single_ typical image, and repeat for each typical image to create a `numpy` array of distances having shape (10000, 10).  You should then find the `np.argmin()` function helpful in finding the predicted label for each validation image.  

In [ ]:
# predicted_classes = return [0]*len(validation_images)

grader.score('tf__smallest_delta', predicted_classes)

## Softmax model

Looking only at the smallest distance is throwing some information away.  We would expect some classes to have more variation that others.  Also, we would expect correlation between the classes&mdash;a small distance to *cat* is probably more likely for *dog* than for *airplane*.

Instead of a trying to work out these effects heuristically, let the computer do it.  Build a softmax model that takes as input features the distances from all ten typical images and makes a class prediction from those.  Again, use this to predict the class for each of the validation images.

**Hint:**
- The labels are given as integers, but softmax expects one-hot encoding of the labels.  The `tf.one_hot` function can do the conversion.

In [ ]:
class LogisticRegression():
    def __init__(self, eta=.1):
        self.opt = tf.keras.optimizers.SGD(learning_rate=eta)
    
    def _logits(self, X):
        return tf.matmul(X, self.W) + self.b
    
    def loss(self, X, y, return_func=False):
        def loss_():
            return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=self._logits(X), labels=y))
        
        if not return_func:
            return loss_()
        
        return loss_
    
    def fit(self, X, y, steps=100000):
        if not hasattr(self, 'W'):
            self.W = tf.Variable(tf.zeros((X.shape[1], 10), dtype=X.dtype))
        if not hasattr(self, 'b'):
            self.b = tf.Variable(tf.zeros((1,10), dtype=X.dtype))
        
        for _ in range(steps):
            self.opt.minimize(self.loss(X, y, return_func=True), [self.W, self.b])
    
    def predict_proba(self, X):
        return tf.nn.softmax(self._logits(X))
        
    def predict(self, X):
        return tf.argmax(self.predict_proba(X), axis = 1)
    
    def score(self, X, y):
        return tf.reduce_mean(tf.cast(tf.equal(self.predict(X), tf.argmax(y,axis=1)), tf.float64))

In [ ]:
model = LogisticRegression()

In [ ]:
X_train = []
for im in train_images:
    im = tf.cast(im, tf.float32)
    diff = np.asarray([c_delta(typical_ims[i],im).numpy() for i in range(10)])
    X_train.append(diff) 
X_train = np.asarray(X_train)

In [ ]:
X_train.shape

In [ ]:
y_train = tf.one_hot(train_labels, 10).numpy()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
X_test = []
for im in validation_images:
    im = tf.cast(im, tf.float32)
    diff = np.asarray([c_delta(typical_ims[i],im).numpy() for i in range(10)])
    X_test.append(diff) 
X_test = np.asarray(X_test)

In [ ]:
y_test = tf.one_hot(train_labels[40000:], 10).numpy()

In [ ]:
model.score(X_test,y_test)

In [ ]:
predicted_classes = model.predict(X_test)
predicted_classes = list(predicted_classes.numpy())

In [ ]:
# predicted_classes = return [0]*len(validation_images)

grader.score('tf__softmax', predicted_classes)

**Extension:** Does this perform any better than the previous solution?  What metric is the right one to use to make this judgment?

## Fully-connected model

The previous model still makes the assumption that this distance metric is the right measure to use.  In this problem, we'll give this up.  Build a multi-layer fully-connected neural network that takes the pixel values as input and yields a class prediction as output.

**Hints:**
- We found that adding more layers didn't help too much.
- Watch out for overfitting.  Dropout can help with this.
- The reference solution achieves an accuracy of about 44% on a training set and 41% on a test set.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
X_train = [np.concatenate(np.concatenate(im)) for im in train_images]

In [ ]:
y_train = train_labels[:49000]

X_t = np.asarray(X_train[:49000])

X_test = np.asarray(X_train[49000:])
y_test = train_labels[49000:]
                          
from tensorflow.keras.utils import to_categorical as one_hot

y_train_hot = one_hot(y_train)
y_test_hot = one_hot(y_test)
                          

N_PIXELS= 32 * 32 * 3
N_CLASSES = 10

hidden_size = 64

In [ ]:
model = keras.models.Sequential()

model.add(
    keras.layers.Dense(
        hidden_size,
        activation='sigmoid',
        use_bias=True,  # The default
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=N_PIXELS**-0.5)
    )
)

model.add(
    keras.layers.Dense(
        10,
        activation='softmax',
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=hidden_size**-0.5)
    )
)


model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.5),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_t, y_train_hot,
                    epochs=30,
                    batch_size=128,
                    validation_data=(X_test,y_test_hot))

In [ ]:
val = [np.concatenate(np.concatenate(im)) for im in validation_images]

In [ ]:
predicted_classes = list(np.argmax(model.predict(np.asarray(val)),axis=1))

In [ ]:
# predicted_classes = return [0]*len(validation_images)

grader.score('tf__fully_connected', predicted_classes)

## Convolutional model

Convolutional neural networks have had a lot of success in image classification.  Build a neural network with convolutional layers to improve the performance.

**Hints:**
- The reference solution uses two convolutional layers and two fully-connected layers.
- We found success with the `AdamOptimizer`.
- The reference solution achieves an accuracy of roughly 80% on a training set and 70% on a test set.

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from random import shuffle
X_train = [np.concatenate(np.concatenate(im)) for im in train_images]

In [ ]:
y_train = train_labels[:48000]

X_t = np.asarray(X_train[:48000])

X_test = np.asarray(X_train[48000:])
y_test = train_labels[48000:]
                          
from tensorflow.keras.utils import to_categorical as one_hot

y_train_hot = one_hot(y_train)
y_test_hot = one_hot(y_test)
                          

In [ ]:
img_size = 32
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)

n_classes = 10
n_channels = 1
filt_size = [5, 5] # 5x5 pixel filters

batch_size = 128
n_epochs = 30

In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Reshape([img_size, img_size, 3]))
model.add(keras.layers.Conv2D(16, filt_size, padding='same',
                              activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2),
                                    padding='same'))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(200, activation='relu'))

model.add(keras.layers.Dense(n_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_t, y_train_hot,
                    epochs=n_epochs,
                    batch_size=batch_size,
                    validation_data=(X_test,y_test_hot))

In [ ]:
val = [np.concatenate(np.concatenate(im)) for im in validation_images]

In [ ]:
predicted_classes = list(np.argmax(model.predict(np.asarray(val)),axis=1))

In [ ]:

# predicted_classes = return [0]*len(validation_images)

grader.score('tf__convolutional', predicted_classes)

## Transfer learning

In transfer learning, we use a network trained on one data set to provide a starting point for the modeling of other data.  As we are trying to model color images, we should look for another network trained on color images.  Luckily, we have already discussed such a network: the Inception network used in the Deep Dream notebook.

The following cell will load the model, omitting its classification layer (since we're not interested in classifying `ImageNet` images).

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
# include_top=False will discard avg_pool before prediction layer
inception = tf.keras.applications.inception_v3.InceptionV3(include_top=True, input_shape=(299, 299, 3))
inception = tf.keras.Model([inception.input], [inception.layers[-2].output]) # manually discard prediction layer

In a transfer learning setup, we will use the first part of the trained network to process the original images, and then train a network to make predictions from the output of the trained network.  There are several ways to accomplish this.

One approach is to connect the new layers to the output of the existing layers.  Data will flow through the pre-trained layers as well as those added.  In the training step, only the new layers should be marked as trainable.

However, this can be a bit wasteful when multiple epochs of training will be undertaken since we recalculate the latent vectors on every training step.  With a smaller data set, such as this, it can be more efficient to pre-calculate the *latent vectors* that are the output of the pre-trained network.  These can be stored and used as input for training a smaller, separate network to make the predictions.  We recommend this approach for this miniproject.

Images should be fed to the `inception` network and then vectorized (you might want to refer to the `TF_DeepDream.ipynb` notebook).

We've loaded the `inception` network with its native image shape: $299 \times 299$.  This implies that we need to upscale our images from $32\times32$ to $299\times299$.  There are a number of ways to do this; the reference solution uses `tf.image.resize` with bilinear interpolation.  (More sophisticated resizing methods produce better results, but will take significantly longer!)

It may make sense to do the rescaling and latent vector calculation at the same time, to avoid storing the (somewhat large) rescaled images unnecessarily.  You also probably want to save those latent vectors to disk, to avoid the need to repeat this calculation later.

**Hints:**
- Be sure to batch this calculation; resizing all 50,000 images at once will cause memory errors.
- The latent vector calculation took us between 30 minutes and 2 hours on a single machine.  You might consider distributing the calculation.
- The latent vectors for the first 10 images have an average of 1983 non-zero values and an overall average value of 0.319.

With the latent vectors calculated, we can use them as input to train a small neural network to make the final predictions.

**Hints:**
- The reference solution has three layers.
- The reference solution achieves a training accuracy of 87% and a test accuracy of 85%.

In [ ]:
for i in range(20):
    images = train_images[i*2500: (i+1)*2500]
    lis = [tf.image.resize(im,[299, 299]).numpy().astype(np.float32) for im in images]
    res = [inception.predict(np.expand_dims(l, axis=0)) for l in lis]
    res_ready = np.asarray(res)
    with open('resize{}.npy'.format(i), 'wb') as f:
        np.save(f, res_ready)
    del res_ready
    del images
    del lis
    del res

In [ ]:
#load all files together
ns = []
for i in range(20):
    a = np.load(open("resize{}.npy".format(i),'rb'))
    ns.extend(a)#.reshape(-1,2048))

In [ ]:
latent_vects = np.concatenate(ns)

In [ ]:
X_train = ns

In [ ]:
y_train = train_labels[:48000]

X_t = np.concatenate(X_train[:48000])

X_test = np.concatenate(X_train[48000:])
y_test = train_labels[48000:]
                          
from tensorflow.keras.utils import to_categorical as one_hot

y_train_hot = one_hot(y_train)
y_test_hot = one_hot(y_test)

In [ ]:
N_PIXELS= 299 * 299 * 3
N_CLASSES = 10

hidden_size = 64

In [ ]:
X_test.shape

In [ ]:
model = keras.models.Sequential()

model.add(
    keras.layers.Dense(
        hidden_size,
        activation='sigmoid',
        use_bias=True,  # The default
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=N_PIXELS**-0.5)
    )
)

model.add(
    keras.layers.Dense(
        10,
        activation='softmax',
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=hidden_size**-0.5)
    )
)


model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.5),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_t, y_train_hot,
                    epochs=2,
                    batch_size=128,
                    validation_data=(X_test,y_test_hot))

In [ ]:
lis = [tf.image.resize(im,[299, 299]).numpy().astype(np.float32) for im in validation_images[:10]]

In [ ]:
for i in range(10):
    images = validation_images[i*1000: (i+1)*1000]
    lis = [tf.image.resize(im,[299, 299]).numpy().astype(np.float32) for im in images]
    res = [inception.predict(np.expand_dims(l, axis=0)) for l in lis]
    res_ready = np.asarray(res)
    with open('test{}.npy'.format(i), 'wb') as f:
        np.save(f, res_ready)

In [ ]:
test = []
for i in range(10):
    a = np.load(open("test{}.npy".format(i),'rb'))
    test.extend(a)#.reshape(-1,2048))

In [ ]:
val = np.concatenate(test)

In [ ]:
predicted_classes = list(np.argmax(model.predict(val),axis=1))

In [ ]:
# predicted_classes = return [0]*len(validation_images)

grader.score('tf__transfer_learning', predicted_classes)

*Copyright &copy; 2020 Pragmatic Institute. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*